# 1 - Simple Sentiment Analysis

Tutorial from :

https://github.com/bentrevett/pytorch-sentiment-analysis

https://www.cnblogs.com/linzhenyu/p/13277552.html

https://www.youtube.com/watch?v=9aYuQmMJvjA&t=839s&ab_channel=sentdex

Dataset Download from :

https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
import torch
from torchtext.legacy import data
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
csv_data = pd.read_csv('MovieReview.csv')
len_train = int(len(csv_data)*0.5)
df_train, df_test = csv_data.iloc[:len_train], csv_data.iloc[len_train:]

TEXT = data.Field(tokenize='spacy', tokenizer_language = 'en_core_web_sm')
LABEL = data.LabelField(dtype = torch.float)


class MyDataset(data.Dataset):
    def __init__(self, csv_data, text_field, label_field, test=False, aug=False, **kwargs):
        
        # Data processing operation format
        fields = [("text", text_field), ("label", label_field)]
        
        examples = []
        if test:
            # If it is a test set, the label is not loaded
            for text in tqdm(csv_data['review']):
                examples.append(data.Example.fromlist([text, None], fields))
        else:
            for text, label in tqdm(zip(csv_data['review'], csv_data['sentiment'])):
                # Enhance Data
                if aug:
                    rate = random.random()
                    if rate > 0.5:
                        text = self.dropout(text)
                    else:
                        text = self.shuffle(text)
                examples.append(data.Example.fromlist([text, label], fields))
                
        super(MyDataset, self).__init__(examples, fields)

        
train_data = MyDataset(df_train, TEXT, LABEL)
test_data = MyDataset(df_test, TEXT, LABEL, True)
print(vars(train_data.examples[0]))
print('------------------------')
print(vars(test_data.examples[0]))

12500it [00:11, 1065.98it/s]
100%|██████████████████████████████████████████████████████████████████████████| 12500/12500 [00:08<00:00, 1517.22it/s]

{'text': ['With', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'MJ', 'i', "'ve", 'started', 'listening', 'to', 'his', 'music', ',', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', ',', 'watched', 'The', 'Wiz', 'and', 'watched', 'Moonwalker', 'again', '.', 'Maybe', 'i', 'just', 'want', 'to', 'get', 'a', 'certain', 'insight', 'into', 'this', 'guy', 'who', 'i', 'thought', 'was', 'really', 'cool', 'in', 'the', 'eighties', 'just', 'to', 'maybe', 'make', 'up', 'my', 'mind', 'whether', 'he', 'is', 'guilty', 'or', 'innocent', '.', 'Moonwalker', 'is', 'part', 'biography', ',', 'part', 'feature', 'film', 'which', 'i', 'remember', 'going', 'to', 'see', 'at', 'the', 'cinema', 'when', 'it', 'was', 'originally', 'released', '.', 'Some', 'of', 'it', 'has', 'subtle', 'messages', 'about', 'MJ', "'s", 'feeling', 'towards', 'the', 'press', 'and', 'also', 'the', 'obvious', 'message', 'of', 'drugs', 'are', 'bad', "m'kay", '.', ' ', 'Visually', 'impressive', 'but'

In [3]:
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
train_data, valid_data = train_data.split(random_state = random.seed(SEED))

print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 8750
Number of validation examples: 3750
Number of testing examples: 12500


In [4]:
TEXT.build_vocab(train_data)
print(len(TEXT.vocab))

65699


In [5]:
MAX_VOCAB_SIZE = 2500

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 2502
Unique tokens in LABEL vocabulary: 2


In [6]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 102437), (',', 97984), ('.', 96928), ('and', 55973), ('a', 55272), ('of', 50714), ('to', 47735), ('is', 38792), ('in', 31316), ("'", 30344), ('I', 28467), ('it', 27251), ('that', 24835), ("'s", 21809), ('this', 21476), ('-', 19164), ('was', 17744), (' ', 15737), ('The', 15583), ('as', 15331)]


We can also see the vocabulary directly using either the `stoi` (**s**tring **to** **i**nt) or `itos` (**i**nt **to**  **s**tring) method.

In [7]:
print(TEXT.vocab.itos[:22])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', "'", 'I', 'it', 'that', "'s", 'this', '-', 'was', ' ', 'The', 'as']


We can also check the labels, ensuring 0 is for negative and 1 is for positive.

In [8]:
print(LABEL.vocab.stoi)

defaultdict(None, {'Positive': 0, 'Negative': 1})


The final step of preparing the data is creating the iterators. We iterate over these in the training/evaluation loop, and they return a batch of examples (indexed and converted into tensors) at each iteration.

We'll use a `BucketIterator` which is a special type of iterator that will return a batch of examples where each example is of a similar length, minimizing the amount of padding per example.

We also want to place the tensors returned by the iterator on the GPU (if you're using one). PyTorch handles this using `torch.device`, we then pass this device to the iterator.

In [9]:
torch.cuda.device_count()

0

In [10]:
BATCH_SIZE = 64

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU')
else:
    device = torch.device('cpu')
    print('Using CPU')
    
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,
    sort=False)


Using CPU


In [11]:
for batch in train_iterator:
    print(batch.text, batch.text.shape)
    print(batch.label, batch.label.shape)
    break

tensor([[  0, 670, 144,  ...,  58,  58, 137],
        [  0,   7,  64,  ...,  50,  23,   0],
        [ 13,  41,  92,  ...,   8, 513, 962],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]]) torch.Size([725, 64])
tensor([1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0.,
        1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1.,
        1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
        1., 0., 0., 1., 0., 1., 0., 1., 0., 0.]) torch.Size([64])


In [12]:
for batch in valid_iterator:
    print(batch.text, batch.text.shape)
    print(batch.label, batch.label.shape)
    break

tensor([[  0,  20,  12,  ...,  12,  12, 520],
        [  0, 249, 567,  ..., 413,  72,  92],
        [  0,  18,  16,  ..., 275, 454,  12],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]]) torch.Size([1157, 64])
tensor([1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1.,
        1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 1., 1., 1., 1., 0., 0.]) torch.Size([64])


In [13]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim) # input layer
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim) # hidden layer
        
        self.fc = nn.Linear(hidden_dim, output_dim) # output layer
        
    def forward(self, text):
        
        '''
        text torch.Size([998, 64])
        embedded torch.Size([998, 64, 100])
        output torch.Size([998, 64, 256])
        hidden torch.Size([1, 64, 256])
        '''
    
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
       
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
       
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

We now create an instance of our RNN class. 

The input dimension is the dimension of the one-hot vectors, which is equal to the vocabulary size. 

The embedding dimension is the size of the dense word vectors. This is usually around 50-250 dimensions, but depends on the size of the vocabulary.

The hidden dimension is the size of the hidden states. This is usually around 100-500 dimensions, but also depends on factors such as on the vocabulary size, the size of the dense vectors and the complexity of the task.

The output dimension is usually the number of classes, however in the case of only 2 classes the output value is between 0 and 1 and thus can be 1-dimensional, i.e. a single scalar real number.

In [14]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

Let's also create a function that will tell us how many trainable parameters our model has so we can compare the number of parameters across different models.

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 342,105 trainable parameters


## Train the Model

In [16]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

#define our loss function
criterion = nn.BCEWithLogitsLoss()

The loss function here is _binary cross entropy with logits_. 

Our model currently outputs an unbound real number. As our labels are either 0 or 1, we want to restrict the predictions to a number between 0 and 1. We do this using the _sigmoid_ or _logit_ functions. 

We then use this this bound scalar to calculate the loss using binary cross entropy. 

The `BCEWithLogitsLoss` criterion carries out both the sigmoid and the binary cross entropy steps.

Using `.to`, we can place the model and the criterion on the device(GPU or CPU)

In [17]:
model = model.to(device)
criterion = criterion.to(device)

Our criterion function calculates the loss, however we have to write our function to calculate the accuracy. 

This function first feeds the predictions through a sigmoid layer, squashing the values between 0 and 1, we then round them to the nearest integer. This rounds any value greater than 0.5 to 1 (a positive sentiment) and the rest to 0 (a negative sentiment).

We then calculate how many rounded predictions equal the actual labels and average it across the batch.

In [18]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

The `train` function iterates over all examples, one batch at a time. 

`model.train()` is used to put the model in "training mode", which turns on _dropout_ and _batch normalization_. Although we aren't using them in this model, it's good practice to include it.

For each batch, we first zero the gradients. Each parameter in a model has a `grad` attribute which stores the gradient calculated by the `criterion`. PyTorch does not automatically remove (or "zero") the gradients calculated from the last gradient calculation, so they must be manually zeroed.

We then feed the batch of sentences, `batch.text`, into the model. Note, you do not need to do `model.forward(batch.text)`, simply calling the model works. The `squeeze` is needed as the predictions are initially size _**[batch size, 1]**_, and we need to remove the dimension of size 1 as PyTorch expects the predictions input to our criterion function to be of size _**[batch size]**_.

The loss and accuracy are then calculated using our predictions and the labels, `batch.label`, with the loss being averaged over all examples in the batch.

We calculate the gradient of each parameter with `loss.backward()`, and then update the parameters using the gradients and optimizer algorithm with `optimizer.step()`.

The loss and accuracy is accumulated across the epoch, the `.item()` method is used to extract a scalar from a tensor which only contains a single value.

Finally, we return the loss and accuracy, averaged across the epoch. The `len` of an iterator is the number of batches in the iterator.

You may recall when initializing the `LABEL` field, we set `dtype=torch.float`. This is because TorchText sets tensors to be `LongTensor`s by default, however our criterion expects both inputs to be `FloatTensor`s. Setting the `dtype` to be `torch.float`, did this for us. The alternative method of doing this would be to do the conversion inside the `train` function by passing `batch.label.float()` instad of `batch.label` to the criterion. 

In [19]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
               
        predictions = model(batch.text).squeeze(1) # feed forward

        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
      
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

`evaluate` is similar to `train`, with a few modifications as you don't want to update the parameters when evaluating.

`model.eval()` puts the model in "evaluation mode", this turns off _dropout_ and _batch normalization_. Again, we are not using them in this model, but it is good practice to include them.

No gradients are calculated on PyTorch operations inside the `with no_grad()` block. This causes less memory to be used and speeds up computation.

The rest of the function is the same as `train`, with the removal of `optimizer.zero_grad()`, `loss.backward()` and `optimizer.step()`, as we do not update the model's parameters when evaluating.

In [20]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

We'll also create a function to tell us how long an epoch takes to compare training times between models.

In [21]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

We then train the model through multiple epochs, an epoch being a complete pass through all examples in the training and validation sets.

At each epoch, if the validation loss is the best we have seen so far, we'll save the parameters of the model and then after training has finished we'll use that model on the test set.

In [22]:
N_EPOCHS = 3

best_valid_loss = float('inf') # infinite large number use for compare and find the minimun value

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 4m 53s
	Train Loss: 0.695 | Train Acc: 50.66%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
Epoch: 02 | Epoch Time: 4m 43s
	Train Loss: 0.693 | Train Acc: 50.39%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%
Epoch: 03 | Epoch Time: 4m 38s
	Train Loss: 0.693 | Train Acc: 50.63%
	 Val. Loss: 0.693 |  Val. Acc: 50.29%


The loss is not really decreasing and the accuracy is poor. This is due to several issues with the model.

Finally, the metric we actually care about, the test loss and accuracy, which we get from our parameters that gave us the best validation loss.

In [23]:
model.load_state_dict(torch.load('tut1-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

KeyError: None